In [1]:
import sys
import os
import warnings
import multiprocessing as mp
import pandas as pd
import numpy as np
import time
import pickle
import datetime

fasta = "/global/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel/magi/cdhitout.faa"
compounds = "/global/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel/Pactolus_Results_20160824_KBL_SolarPanel_MP/pactolus_hits.csv"
gene_to_reaction = "/global/project/projectdirs/openmsi/projects/temp_chem_net_data/MAGI_data/cdhitout_20170531/gene_to_reaction.pkl"
output = "/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel/magi_lipid_posneg"
reaction_to_gene = "/global/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel/magi_lipid_posneg/reaction_to_gene.pkl"
compound_to_reaction = "/global/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel/magi_lipid_posneg/compound_to_reaction.pkl"

max_cpu = mp.cpu_count()
warnings.filterwarnings('ignore')

# load local settings
sys.path.insert(
    0,
    '/project/projectdirs/metatlas/projects/metatlas_reactions/')
from local_settings import local_settings as settings_loc
my_settings = getattr(
    __import__(
        'local_settings',
        fromlist=[settings_loc.SETTINGS_FILE]), settings_loc.SETTINGS_FILE)

# import workflow helpers after all the argument checking
sys.path.insert(
    0,
    os.path.join(my_settings.repo_location, 'workflow/helpertools'))
import workflow_helpers as mg

# path to MAGI data storage
MAGI_PATH = my_settings.magi_results_storage
experiment_path = output
if not os.path.isdir(experiment_path):
    os.makedirs(experiment_path)



loading refseq and reaction tables...
Reference sequences in this file: /project/projectdirs/metatlas/projects/metatlas_reactions/workflow/database/mrs_reaction_filtered_refseq_db_newrxns_actinofixed.pkl
135905 reference sequences
MRS-Reaction: /project/projectdirs/metatlas/projects/metatlas_reactions/workflow/database/mrs_reaction_newrxns_added_actinofix.pkl
16966 reactions
10368 reactions with a refseq
9363 reactions with an EC
loading compound table
loading chemnet files
All databases loaded into memory


In [2]:
# load genome
genome, genome_db_path = mg.load_genome(fasta, MAGI_PATH, annotation_file=None)

# load pactolus results
compounds = mg.load_dataframe(compounds)
# auto-rename pactolus columns
# if pactolus:
#     compounds = mg.reformat_pactolus(compounds)
# remove any missing compounds
compounds = compounds[~pd.isnull(compounds['original_compound'])]
compounds.fillna('', inplace=True)

u_cpds = compounds['original_compound'].unique()
compounds['compound_score'] = compounds['compound_score'].apply(float)
gene_to_reaction_top = pd.read_pickle(gene_to_reaction)
compound_to_reaction = pd.read_pickle(compound_to_reaction)
reaction_to_gene_top = pd.read_pickle(reaction_to_gene)


1804258 genes
saved gene_sequence table here: /global/project/projectdirs/openmsi/projects/temp_chem_net_data/MAGI_data/gene_fastas/cdhitout_sequences.pkl
blast database stored here: /global/project/projectdirs/openmsi/projects/temp_chem_net_data/MAGI_data/BLAST_dbs/cdhitout.db


# Remove any compounds that don't have a reaction

In [9]:
compound_to_reaction.columns

Index([u'Unnamed: 0', u'spectrum_index', u'polarity', u'precursor intensity',
       u'precursor_mz', u'retention_time', u'filename_x', u'compound_score',
       u'compound', u'compound_index', u'filename_y', u'inchi',
       u'original_compound', u'ms1_mass', u'permanent_charge',
       u'mono_isotopic_molecular_weight', u'creation_time', u'synonyms',
       u'neutralized_2d_inchi_key', u'chebi_url', u'img_abc_id',
       u'neutralized_2d_inchi', u'lipidmaps_url', u'source', u'kegg_url',
       u'hmdb_url', u'wikipedia_url', u'formula', u'number_components',
       u'iupac_name', u'username', u'pubchem_compound_id', u'description',
       u'metacyc_id', u'kegg_id', u'hmdb_id', u'chebi_id',
       u'neutralized_inchi_key', u'neutralized_inchi', u'name',
       u'num_free_radicals', u'lipidmaps_id', u'last_modified', u'pubchem_url',
       u'unique_id', u'detected_polarity', u'ppm', u'level', u'neighbor',
       u'note', u'reaction_id'],
      dtype='object')

In [4]:
print compound_to_reaction.shape
idx1 = compound_to_reaction.level == 0
idx2 = compound_to_reaction.reaction_id != ''


(6903869, 51)

In [17]:
compound_to_reaction = compound_to_reaction[(idx1 & idx2)]
print compound_to_reaction.shape

(179366, 51)


In [18]:
%%time
# from dask import dataframe as dd
import dask.dataframe as dd
#Construct a dask objects from a pandas objects
left1 = dd.from_pandas(compound_to_reaction, npartitions=8)
right1 = dd.from_pandas(reaction_to_gene_top, npartitions=4)

#merge on key
compound_to_gene = dd.merge(left1, right1, on='reaction_id', how='left').compute()

CPU times: user 19 s, sys: 4.18 s, total: 23.2 s
Wall time: 22.8 s


In [19]:
compound_to_gene_small = compound_to_gene[['subject acc.', 'reaction_id',
                            'e_score', 'compound_score',
                            'original_compound', 'level', 'neighbor',
                            'note']]


In [20]:
# okay to drop duplicates, because i only care about these columns 
# anyway; if these are duplicated then other information doesn't really 
# matter or can easily be re-expanded by joining 
compound_to_gene_small.drop_duplicates(inplace=True)

gene_to_reaction_small = gene_to_reaction_top[['query acc.', 'reaction_id',
                                                'e_score']]
gene_to_reaction_small.drop_duplicates(inplace=True)


In [21]:
del reaction_to_gene_top
del compound_to_reaction
del compound_to_gene

In [22]:
%%time
# from dask import dataframe as dd
import dask.dataframe as dd
#Construct a dask objects from a pandas objects
left1 = dd.from_pandas(compound_to_gene_small, npartitions=8)
right1 = dd.from_pandas(gene_to_reaction_small, npartitions=8)


# Make an integrated dataframe, joining on the gene
df = dd.merge(left1, right1, 
    left_on='subject acc.', right_on='query acc.', 
    suffixes=('_r2g', '_g2r'), how='outer').compute()

CPU times: user 23.3 s, sys: 5.44 s, total: 28.8 s
Wall time: 27 s


In [23]:
del left1
del right1

In [24]:
# %%time
# #think about saving df here
# df.to_hdf('/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel/df_magi_save.h5',
#     'merged_before_score', mode='w', format='table',
#     complib='blosc', complevel=9)

In [25]:
%%time
df.reset_index(inplace=True, drop=True)

CPU times: user 1e+03 µs, sys: 12 ms, total: 13 ms
Wall time: 12.4 ms


In [26]:
# df.columns

In [27]:
# %%time
# #should take about 35 minutes to make the dask dataframe
# #partitions: 1 = 52
# #partitions: 4 = 1min 55 s
# # df.drop_duplicates(inplace=True)
# print df.shape
# ddf = dd.from_pandas(df.head(2000000), npartitions=1)


In [28]:
# %%time
# df2 = ddf.drop_duplicates().compute()

# print df2.shape

In [29]:
%%time
# Clean up reaction_id_r2g column
df.reaction_id_r2g = df.reaction_id_r2g.replace('', np.nan)

CPU times: user 5.33 s, sys: 475 ms, total: 5.8 s
Wall time: 5.8 s


In [30]:
df.shape

(22008772, 11)

In [31]:
%%time
df['reaction_id_r2g'] = df['reaction_id_r2g'].astype(float)

CPU times: user 119 ms, sys: 98 ms, total: 217 ms
Wall time: 214 ms


In [32]:
%%time
# Clean up NaNs in string columns that make column object type
def check_str(x):
    if isinstance(x, str):
        return True
    else:
        return False

for c in df.columns:
    print c
    if len(df[c].apply(type).unique()) > 1:
        print c,'is too many'
        string_checked = df[c].apply(check_str)
        if string_checked.any():
            df[c].fillna('', inplace=True)

# Clean up neighbor column
df['neighbor'] = df['neighbor'].astype(str)

subject acc.
subject acc. is too many
reaction_id_r2g
e_score_r2g
compound_score
original_compound
original_compound is too many
level
neighbor
neighbor is too many
note
note is too many
query acc.
query acc. is too many
reaction_id_g2r
e_score_g2r
CPU times: user 1min 10s, sys: 2.17 s, total: 1min 12s
Wall time: 1min 12s


In [33]:
%%time
# score reciprocal agreement
df = mg.reciprocal_agreement(df)

CPU times: user 20 s, sys: 8.67 s, total: 28.6 s
Wall time: 28.6 s


In [34]:
%%time
# calculate homology score
score = mg.homology_score(df)
# the nulls get a really low score
score[pd.isnull(score)] = 1
df['homology_score'] = score

CPU times: user 275 ms, sys: 259 ms, total: 534 ms
Wall time: 548 ms


In [35]:
%%time
# adjust compound score slightly by leveled search
df['level_adjusted_compound_score'] = df['compound_score'].values / \
                                            (df['level'].values + 1)

CPU times: user 116 ms, sys: 246 ms, total: 362 ms
Wall time: 360 ms


In [36]:
# reaction connection score says if the compound got connected to any
# reaction in the database. Can't have zero because that messes up
# geometric mean, so added a small number.
df['reaction_connection'] = df[['reaction_id_r2g', 'reaction_id_g2r']]\
                                .apply(pd.notnull).sum(axis=1) + 0.01

In [37]:
# calculate final MAGI integrated score
scoring_data = ['level_adjusted_compound_score', 'reciprocal_score', \
                'homology_score', 'reaction_connection']
scores = []
to_score = df[scoring_data].values
data = []
for s in to_score:
    data.append(mg.magi_score(s, weights=None))
scores.append(data)
df['MAGI_score'] = scores[0] / (4. ** df['level'].values)

In [38]:
# find gene ids that are floats, convert those to strings, without the decimal
float_entries = df['subject acc.'].apply(lambda x: isinstance(x, float))
df.loc[float_entries, 'subject acc.'] = df.loc[float_entries, \
                'subject acc.'].apply(lambda x: "{:.0f}".format(x))

In [39]:
# sort the final table and drop key duplicates
df = df.sort_values(
    ['original_compound', 'MAGI_score'], 
    ascending=[True, False]
    ).drop_duplicates(
        ['original_compound', 'level', 'neighbor', 'compound_score',
         'reciprocal_score', 'query acc.', 'reaction_id_r2g',
         'reaction_id_g2r']
         )

In [40]:
df.to_hdf(os.path.join(experiment_path, 'merged_with_score.h5'),
    'merged_with_score', mode='w', format='table',
    complib='blosc', complevel=9)

In [41]:
# save the full dataframe
df.to_csv(os.path.join(experiment_path, 'magi_results.csv'))

In [42]:
# save a compound-centric dataframe, where only the best row for each
# original_compound was chosen (this is only for compound scoring, do
# not use this for any kind of gene function analysis!)
compound_centric = df[pd.notnull(df['original_compound'])]\
                     .sort_values('MAGI_score', ascending=False)\
                     .drop_duplicates(['original_compound', 'compound_score'])
compound_centric = pd.merge(
    compound_centric, compounds,
    on=['original_compound', 'compound_score'],
    how='right')

In [43]:
compound_centric.to_csv(os.path.join(experiment_path, 
    'magi_compound_results.csv'))

# Show taxonomic groups as rows and compounds as columns. Cluster the taxonomic groups and compounds